In [ ]:
# b9a44c5c-ec32-467d-8c4b-534b3f2ea9db is my demo tenant
$TenantId = 'b9a44c5c-ec32-467d-8c4b-534b3f2ea9db' #"<TENANT_ID>"

. "..\src\functions.ps1"

try{
    Set-AzContext -TenantId $TenantId -ErrorAction Stop
}
catch{
    Connect-AzAccount -TenantId $TenantId
}

# Use HTTPS only

*Severity*: High

Configure App Service to use HTTPS only. This causes App Service to redirect from HTTP to HTTPS.  Strongly consider the use of HTTP Strict Transport Security (HSTS) in your code or from your WAF, which informs browsers that the site should only be accessed using HTTPS.

It can be *acceptable* to not enforce HTTPS in cases such as the site being behind Azure Front Door via a private endpoint.

The overhead from HTTPS compared to HTTP is mostly some handshakes and generally has no performance impact.

The following sites are not enforcing HTTPS.

In [ ]:
$CheckListId = "475ba18f-dbf5-490c-b65d-e8e03f9bcbd4"

Invoke-AzResourceGraphCheck -CheckList $appsvcSecurityChecklist -CheckListId $CheckListId -DebugMode $DebugMode


# Set minimum TLS policy to 1.2

*Severity*: Medium

Set minimum TLS policy to 1.2 in App Service configuration.

Only legacy clients are not able to support TLS 1.2, and it should generally be safe to enforce using TLS 1.2 or higher.

The following sites are not enforcing the use of TLS 1.2 or higher.

In [ ]:
$CheckListId = "c115775c-2ea5-45b4-9ad4-8408ee72734b"

Invoke-AzResourceGraphCheck -CheckList $appsvcSecurityChecklist -CheckListId $CheckListId -DebugMode $DebugMode


# Turn off remote debugging

*Severity*: High

Remote debugging must not be turned on in production as this opens additional ports on the service which increases the attack surface. Note that the service does turn of remote debugging automatically after 48 hours.

The following sites have remote debugging enabled.

In [ ]:
$CheckListId = "d9bd3baf-cda3-4b54-bb2e-b03dd9a25827"

Invoke-AzResourceGraphCheck -CheckList $appsvcSecurityChecklist -CheckListId $CheckListId -DebugMode $DebugMode


# Subnets should have an NSG associated

Below shows subnets with no NSG associated. Delegated subnets are less important to protect using NSGs, but should still be so if at all possible.

In [ ]:
$Query = @"
resources
| where type == "microsoft.network/virtualnetworks"
| mvexpand subnets = properties.subnets
| extend Delegated = iif(array_length(subnets.properties.delegations) == 0, "no", "yes")
| where isempty(subnets.properties.networkSecurityGroup)
| project ["VNET Name"] = name, ["Subnet Name"] = subnets.name, Delegated, id = subnets.id
"@

Search-AzGraph -Query $Query | Select-Object -ExpandProperty Data | Select-Object -ExcludeProperty ResourceId | Format-Table -AutoSize